# initializing and imports

In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import datetime
import inspect
import re
import time

# غیرفعال کردن هشدار MarkupResemblesLocatorWarning
import warnings
from bs4 import MarkupResemblesLocatorWarning
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)


In [2]:
hunting = 0  #  شیئی گروه برای ارسال پیام های یافت شده
previous_messages = []  #  زمان ارسال پیام های قبل برای جلوگیری از ارسال مجدد آن ها
sending_messages=[]  #  پیام های یافت شده را در این نگه میداریم تا زمان ارسال و بعد مجددا خالی اش می کنیم
# chat_title=''
# chat_id=''
# timestamp = ''
driver = webdriver.Chrome()
driver.get("https://web.telegram.org/k/")

# Text Processing

### text_processing

In [169]:
#  تابع پردازش پیام ها
def text_processing(text):
    if(text == []):
        return False
    positive_expression =['مشاور','مشاوره','وکیل','منصوری','روادید',]

    negetive_expression =['تماس بگیرید','لطفا جهت کسب اطلاعات',
                          'خدمات ما شامل', 'لطفا جهت ثبت نام']
    
    for expression in negetive_expression:
        if expression in text:
            return False
        
    for expression in positive_expression:
        if expression in text:
            return True
    
    return False


### remove_html_and_patterns

In [170]:
def remove_html_and_patterns(text):
    # مرحله 1: حذف تگ‌های HTML
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text()

    # مرحله 2: حذف الگوهای خاص
    patterns = [
        r'\b\w*\.png\b',  # الگو برای *.png
        r'\bclass\w*\b',  # الگو برای class*
        r'\b\w*emoji\w*\b'  # الگو برای *emoji*
    ]
    
    for pattern in patterns:
        clean_text = re.sub(pattern, '', clean_text)

    # مرحله 3: حذف فاصله‌های اضافی
    clean_text = ' '.join(clean_text.split())

    return clean_text

# Dive in HTML

## Finding Things

### is_a_super_group_open

In [171]:
# بررسی می کند که آیا دکمه سوپر گروه باز است و درست یا غلط برمیگرداند
def is_a_super_group_open():
    try:
        if(driver.find_element(By.CSS_SELECTOR, "button.btn-icon.sidebar-close-button").find_element(By.CSS_SELECTOR,"span.tgico.button-icon")):
            return False
    except:
            return True

### find_list_of_all_chats

In [172]:
#  ساختن لیستی از کل چت های موجود    
def find_list_of_all_chats():
    close_super_group_button()
    return driver.find_elements(By.CSS_SELECTOR, "a.row.no-wrap.row-with-padding.row-clickable.hover-effect.rp.chatlist-chat.chatlist-chat-bigger.row-big") 


### find_list_of_all_sub_chats

In [173]:
#  ساختن لیستی از کل زیر چت های موجود    
def find_list_of_all_sub_chats():
    return driver.find_elements(By.CSS_SELECTOR,'ul.chatlist')[1] \
        .find_elements(By.CSS_SELECTOR,"a.row.no-wrap.row-clickable.hover-effect.rp.chatlist-chat.chatlist-chat-bigger.row-big")

### find_chat_title_and_id

In [174]:
#  یافتن عنوان و آیدی چت
def find_chat_title_and_id(chat):
    return [chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title")\
                .text ,\
            chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title") \
                .get_attribute('data-peer-id')]

### find_sub_chat_title_and_id

In [175]:
#  یافتن عنوان و آیدی زیرچت
def find_sub_chat_title_and_id(chat):
    return [chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title")\
                .text ,\
            chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title") \
                .get_attribute('data-peer-id')+\
            chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title") \
                .get_attribute('data-thread-id')]

### is_there_unread_message

In [176]:
#  چک می کند که آیا در این چت پیام خوانده نشده وجود دارد یا خیر
def is_there_unread_message(chat):
    try:
        if(chat.find_element(By.CSS_SELECTOR,"div.dialog-subtitle-badge.badge.badge-22.dialog-subtitle-badge-unread.is-visible.unread").text):
            return True  #  اگر این چت تگ پیام خوانده نشده داشته باشد مقدار تورو را برمگرداند وگرنه مقدار فالس را
    except:
        return False

### check_if_message_is_new

In [177]:
def check_if_message_is_new():
    global previous_messages, chat_id, timestamp
   
    is_new = True # آیا آی دی گروه جدید است؟
    for row in range(len(previous_messages)):
        if(previous_messages[row][0] == chat_id):
            is_new = False
            break
    if(is_new):
        previous_messages.append([])
        previous_messages[len(previous_messages) - 1].append(chat_id)
        row += 1

    for col in range(len(previous_messages[row])):
        if (previous_messages[row][col] == timestamp):
            return False
    
    previous_messages[row].append(timestamp)
    return True

## Do Something

### close_super_group_button

In [178]:
# این تابع دکمه بستن سوپر گروه ها را اگر باز شده باشند می زند تا بسته شود
def close_super_group_button():
        try:
                super_group_button = driver.find_element(By.CSS_SELECTOR, "button.btn-icon.sidebar-close-button")
                super_group_button.click()
        except:
                {}

### scroll_down

In [179]:
# با فراخوانی این تابع دکمه اسکرول داون در صورت وجود فشرده می شود
def scroll_down():
    scroll_down_button = driver.find_element(By.CSS_SELECTOR, ".btn-circle.btn-corner.z-depth-1.bubbles-corner-button.chat-secondary-button.bubbles-go-down.rp")
    try:
        scroll_down_button.click()
        time.sleep(3)
    except:
        {}

### clicking

In [180]:
def clicking(item):
    try:
        item.click()
        return False   # it means there is no problem in clicking the item
    except:
        time.sleep(10)
        try:
            item.click()
            return False   # it means there is no problem in clicking the item
        except:
            return True   # it means there is a problem in clicking the item

# Axiliary Functions

### warm_up

In [181]:
def warm_up():
    global previous_messages
    chats = find_list_of_all_chats()
    for chat in chats:
        [chat_title, chat_id] = find_chat_title_and_id(chat)
        if not(redundent_chats(chat_id)):
            previous_messages.append([chat_id])
            

### redundent_chats

In [182]:
#  چت هایی که مطمئنیم نیازی به بررسی آن ها نیست
def redundent_chats(chat_id):
    if chat_id in ['5427116584','-4214311773']:
        return True
    else:
        return False

### trim_rows

In [183]:
# only keep the last message info
def trim_rows(max_elements=50):
    global previous_messages
    
    for i in range(len(previous_messages)):
        if len(previous_messages[i]) > max_elements:
            # حذف عناصر از عنصر دوم تا تعداد مورد نظر
            previous_messages[i] = [previous_messages[i][0]] + previous_messages[i][len(previous_messages[i]) - max_elements + 1:]

# Comunication

### send_message

In [184]:
def send_message(message):
    close_super_group_button()
    hunting.click()
    time.sleep(1)
    input_box = driver.find_element(By.CSS_SELECTOR, 'div.input-message-input')
    # پاک کردن هر مقدار موجود در اینپوت باکس
    input_box.clear()  # ممکن است این متد بر روی المان‌های contenteditable کار نکند، بسته به نوع مرورگر
    # ارسال مقدار مورد نظر به اینپوت باکس
    input_box.send_keys(message)
    time.sleep(2)
    
    send_button = driver.find_element(By.CSS_SELECTOR, 'div.btn-send-container')  #  انتخاب دکمه ارسال
    send_button.click()
    
    
    # # در صورت نیاز به ارسال اینتر در انتها
    # time.sleep(2)
    # input_box.send_keys(Keys.RETURN)

### log_error

In [185]:
def log_error(error_message):
    caller = inspect.stack()[1]
    try:
        with open('error_log.txt', 'a') as file:
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            file.write(f"Time: {current_time}\n")
            file.write(f"Error in line <<< {caller.lineno} >>> of Function <<< {caller.function } >>>. \n")
            file.write(f"Error: {error_message}\n")
            file.write("\n" + "=*"*75 + "\n")
    except Exception as e:
        print(f"Failed to log error: {e}")


# Main Logic

### survey_the_chat

In [186]:
#  در اینجا ما برای یک چت پیام ها را بررسی می کنیم که آیا موردی که مد نظرمان باشد یافت می شود یا خیر
def survey_the_chat(chat):
    global sending_messages, hunting, chat_id, timestamp
    
    caller_function = inspect.currentframe().f_back.f_code.co_name    
    
    [chat_title, chat_id] = find_chat_title_and_id(chat)
    if chat_id == '-2219036112':   # گروه هانتینگ 
        hunting = chat
        return
    
    if(redundent_chats(chat_id)):
        # print('redundent chat <- <- <- ', chat_title)
        return

    if(not(is_there_unread_message(chat))):
        return;   
                
    # print(f"Time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} necessary chat -> -> -> ", chat_title)         
    scroll_down()
    converstion_list = driver.find_elements(By.CSS_SELECTOR, "div.bubbles-group")   # پیام هر کاربر در تگ با این کلاس هست
    for conversation in converstion_list:
        try:
            user_name = remove_html_and_patterns(conversation.find_element(By.CSS_SELECTOR, "span.peer-title").get_attribute("innerHTML"))
            message = remove_html_and_patterns(conversation.find_element(By.CSS_SELECTOR,"span.translatable-message").get_attribute("innerHTML"))
            timestamp = conversation.find_element(By.CSS_SELECTOR,'.is-in').get_attribute('data-timestamp')
            # data_peer_id = conversation.find_element(By.CSS_SELECTOR,'.is-in').get_attribute('data-peer-id')
        except:
            continue
        if (check_if_message_is_new()):
            if(text_processing(message)):
                for_sending = f" **نام گروه :** <<< {chat_title} >>> **نام کاربر : ** <<< {user_name} >>> **پیام کاربر :** : {message}"
                sending_messages.append(for_sending)
                print(' ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ')
                print(' +++++++++++++++++++++++   WE GOT THE NEW FOLLOWING MESSAGE   +++++++++++++++++++++++++++ ')
                print(' ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ')
                print(for_sending)

### survey_the_sub_chats

In [187]:
def survey_the_sub_chats(mother_chat_title = 'unknown'):
    global sending_messages, chat_id, timestamp
    cat_chats = find_list_of_all_sub_chats()
    for chat in cat_chats:
        if(True): #is_there_unread_message(chat)):              
            [sub_chat_title, chat_id] = find_sub_chat_title_and_id(chat)
            # print(f"Time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ================subgroup {sub_chat_title} of mother group {mother_chat_title}==========")
            chat_title = f"{sub_chat_title} ({mother_chat_title})"
            if clicking(chat):
                continue
            time.sleep(2)  # کمی صبر کنید تا عملیات کلیک و هر اتفاق بعد از آن انجام شود
            scroll_down()
            time.sleep(1)
            sub_converstion_list = driver.find_elements(By.CSS_SELECTOR, "div.bubbles-group") # پیام هر کاربر در تگ با این کلاس هست
            for conversation in sub_converstion_list:
                try:
                    user_name = remove_html_and_patterns(conversation.find_element(By.CSS_SELECTOR,"span.peer-title").text)
                    message = remove_html_and_patterns(conversation.find_element(By.CSS_SELECTOR,"span.translatable-message").text)
                    timestamp = conversation.find_element(By.CSS_SELECTOR,'div.is-in').get_attribute('data-timestamp')
                except Exception as e:
                    # print(f"**************************************Error: {e}")
                    continue
                    
                if (check_if_message_is_new()):
                    if(text_processing(message)):
                        for_sending = f" **نام گروه :** <<< {chat_title} >>> **نام کاربر : ** <<< {user_name} >>> **پیام کاربر :** : {message}"
                        sending_messages.append(for_sending)
    close_super_group_button()
                    

### clean_message

In [188]:
def clean_message(text):
    # Replace all newline characters with a space
    text = text.replace('\n', ' ')
    # Split the text by any whitespace and rejoin with a single space
    cleaned_text = ' '.join(text.split())
    return cleaned_text

### once_check_all_groups

In [189]:
def once_check_all_groups():
    global hunting, sending_messages

    chats = find_list_of_all_chats()
    for chat in chats:
        [chat_title, chat_id] = find_chat_title_and_id(chat)
        chat.click()
        time.sleep(2)
        if  (is_a_super_group_open()):
            survey_the_sub_chats(chat_title)        
        else:
            survey_the_chat(chat)          

    try:
        print(f'               ********************  {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  *************************')
        print('               *************************  END OF LOOP  ****************************')
        print('               ********************************************************************')
        print('')
        for message in sending_messages:
            message = clean_message(message)
            print('the current messages: +++++++++++++++++++++++++    ', message)
            send_message(message)
            print('&&&&&&&&&&&&&&           THAT ONE SENT           &&&&&&&&&&&&&&&')
            print(f"Time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} :\n"+message)
            time.sleep(4)
        if len(sending_messages) == 0:
            print('^^^^^^^^^^^^^^^^^^^^^^   THERE WAS NO MESSAGE TO SENT   ^^^^^^^^^^^^^^^^^^^^^^^^^^')
        else:
            print('^^^^^^^^^^^^^^^^^^^^^^   ALL MESSAGES SUCCESSFULLY SENT   ^^^^^^^^^^^^^^^^^^^^^^^^^^')
            sending_messages = []  #  ریست کردن استورج جمع آوری پیام ها      
    except:
        print('%%%%%%%%%%%%%%%%%%%%%      there was a problem middle sending the messages and now we are in except part         %%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print(' the message with problem is:')
        print(message)
        {} 
    trim_rows()  #  حذف اضافات ماتریس اطلاعات مسج های قبلی


### main

def main():
    # warm_up()
    while True:
        print('***********************  NEW ROUND  ****************************************')
        once_check_all_groups()


# RUNNING TIME

In [190]:
def main():
    # warm_up()
    while True:
        print('               **************************  NEW ROUND  *****************************')

        once_check_all_groups()

if __name__ == "__main__":
    main()

               **************************  NEW ROUND  *****************************


               ********************  2024-06-27 14:39:55  *************************
               *************************  END OF LOOP  ****************************
               ********************************************************************

^^^^^^^^^^^^^^^^^^^^^^   THERE WAS NO MESSAGE TO SENT   ^^^^^^^^^^^^^^^^^^^^^^^^^^
               **************************  NEW ROUND  *****************************


#  تستی 

In [ ]:
sending_messages=[]
# previous_messages=[]

In [ ]:

[chat_title, chat_id] = find_chat_title_and_id(chats[0])

In [ ]:
print(text_processing(message + 'مشاور'))

سلام دوستان وقت بخیر آیا کسی می‌دونه که اگر با ایمیگریشن تماس بگیریم و درخواست تغییر آدرس بکنیم آی...مشاور
True


In [ ]:
# converstion_list[-1]
message = remove_html_and_patterns(converstion_list[-1].find_element(By.CSS_SELECTOR,"span.translatable-message").get_attribute("innerHTML"))